In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.neural_network import MLPRegressor
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [2]:
# new_toll_time = pd.read_csv('new_toll_time_no_0.csv')

In [3]:
# new_toll_time.shape

In [4]:
# new_toll_time.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
toll = pd.read_csv('time_between_two_tolls_new.csv')

In [7]:
toll['ist_timestamp'] = toll['ist_timestamp'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
toll['loading_out_time'] = toll['loading_out_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
toll['hours']  = toll['ist_timestamp'] - toll['loading_out_time']
toll['hours'] = toll.hours.apply(lambda x:x.total_seconds()/3600)

In [8]:
toll['distance_travelled'] = 0.0
toll['time_taken'] = toll['hours']
toll['time_btn_tolls'] = 0

In [9]:
new_time = pd.DataFrame(columns=toll.columns.tolist())

In [10]:
for truck in toll.vehicle_no.unique().tolist():
    for st_time in toll.start_date.unique().tolist():
        df = toll[(toll.vehicle_no == truck) & (toll.start_date == st_time)].sort_values('odometer_km')
        df.distance_travelled = df.odometer_km.diff()
        df.time_btn_tolls = df.hours.diff()
        new_time = new_time.append(df)

In [11]:
new_time = new_time.loc[new_time['distance_travelled'] != 0]
new_time = new_time.fillna(0)

In [12]:
in

In [13]:
new_time.loc[new_time['time_btn_tolls'] == 0, 'time_btn_tolls']  = new_time.loc[new_time['time_btn_tolls'] == 0, 'time_taken']  

In [14]:
today_date = datetime.datetime.now()
today_date.strftime("%Y-%m-%d %H:%M:%S")
today = today_date.strftime("%Y-%m-%d %H:%M:%S")
new_time['today'] = datetime.datetime.strptime(today, '%Y-%m-%d %H:%M:%S')

new_time['start_date'] = new_time['start_date'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_time['eta'] = toll['eta'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )
new_time['unloading_in_time'] = new_time['unloading_in_time'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

new_time['days_from_now'] = new_time['today'] - new_time['start_date'] 
new_time['days_from_now'] = new_time['days_from_now'].apply(lambda x: x.days)

new_time['avg_travel_time'] = new_time['start_date'] + (new_time['unloading_in_time'] - new_time['start_date'])/2

new_time['month_of_travel'] = new_time['avg_travel_time'].apply(lambda x: x.month)

new_time['time_of_travel'] = new_time.ist_timestamp.apply(lambda x: x.hour)

new_time['day_of_travel'] = new_time['avg_travel_time'].apply(lambda x: x.weekday())




In [15]:
toll_freq = dict(new_time['toll_both'].value_counts())
new_time['toll_freq'] = new_time['toll_both']
new_time['toll_freq'] = new_time['toll_freq'].apply(lambda x: toll_freq[x] )
m = int(0.3 * new_time['toll_freq'].median())
new_time = new_time[(new_time['toll_freq'] > m)]
new_time.drop(['toll_freq'], axis=1, inplace=True)

In [17]:
new_time['median_timestamp'] = 0

In [18]:
for toll_b in new_time.toll_both.unique().tolist():
    new_time.loc[new_time['toll_both'] == toll_b, 'median_timestamp'] = new_time.loc[new_time['toll_both'] == toll_b]['time_taken'].median()

In [19]:
new_time['median_timestamp'] = new_time['median_timestamp'].apply(lambda x:datetime.timedelta(hours=x))
new_time['median_timestamp'] = new_time['median_timestamp'] + new_time['loading_out_time']

In [20]:
new_time['delay'] = new_time['ist_timestamp'] - new_time['median_timestamp']

In [21]:
new_time['delay'] = new_time['delay'].apply(lambda x: x.total_seconds()/3600)

In [22]:
new_time['loading_out_time'] = new_time['loading_out_time'].apply(lambda x: x.hour)

In [31]:
final_df = new_time[['toll_both','vehicle_no','loading_out_time','ist_timestamp','distance_travelled','distance_travelled_cummulative','time_of_travel','day_of_travel','month_of_travel','days_from_now','time_taken','delay','time_btn_tolls']]

In [49]:
def find_toll_num(df):
    toll_dict = {}
    new_toll_dict = {}
    x = {}
    num = 1
    for toll in df.toll_both.unique().tolist():
        t = df['distance_travelled_cummulative'][df.toll_both == toll].median()
        toll_dict[toll] = t
    
    for key, value in sorted(toll_dict.iteritems(), key=lambda (k,v): (v,k)):
        x[key] = value
        new_toll_dict[key] = num
        num+=1
        
    return new_toll_dict, x

In [50]:
new_toll_dict,x = find_toll_num(final_df)

In [54]:
sorted_toll_list = []
for key, value in sorted(x.iteritems(), key=lambda (k,v): (v,k)):
    sorted_toll_list.append(key)

In [55]:
sorted_toll_list

['Sergrarh',
 'Panikholi',
 'Manguli',
 'Gudipada (Old Gangapada)',
 'Gurapalli',
 'Bellupada',
 'Madapam',
 'Chilakapalem',
 'Laxmipuram',
 'Nathavalasa/ Vizianagaram',
 'Vempadu',
 'Krishnavaram',
 'Kalaparru',
 'Pottipadu',
 'Kaza',
 'Bolapalli',
 'Tangutur',
 'Sunambatti (Musunur)',
 'Budanam',
 'Gaddurur',
 'Mulbagal']

In [187]:
# dict_toll_both_train_df = change_toll_booth_to_dummy(train_df)
train_df.toll_both = train_df.toll_both.replace(new_toll_dict)
test_df.toll_both = test_df.toll_both.replace(new_toll_dict)

In [188]:
final_test_copy = test_df.copy()
final_train_copy = train_df.copy()
test_df

,toll_both,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,time_taken,delay,time_btn_tolls
561,1,1,0.00,11,5,9,89,33.987222,-2.299028,33.987222
739,2,1,93.71,15,5,9,89,38.447500,-2.596667,4.460278
919,3,1,53.26,18,5,9,89,41.426944,-1.191944,2.979444
383,4,1,56.30,20,5,9,89,43.580000,-0.745833,2.153056
199,5,1,88.58,23,5,9,89,46.131667,-1.620556,2.551667
2159,6,1,83.91,3,5,9,89,50.111111,-4.525278,3.979444
2335,7,1,116.37,8,5,9,89,55.362222,-3.549444,5.251111
1475,8,1,26.47,9,5,9,89,56.087778,-4.153056,0.725556
2518,10,1,39.33,12,5,9,89,59.095278,-2.910556,3.007500
1655,11,1,132.47,17,5,9,89,88.823333,20.011111,29.728056


In [260]:
# def change_vehicle_no_to_dummy(df):
#     num = 1
#     dict_vehicle_no = {}
#     for truck in df['vehicle_no'].unique().tolist():
#         df['vehicle_no'] = df['vehicle_no'].replace({truck: num})
#         dict_vehicle_no[num] = truck
#         num += 1
#     return dict_vehicle_no

In [261]:
# dict_vehicle_no_train_df = change_vehicle_no_to_dummy(final_train_df)
# rev_dict_vehicle_no_test_df = dict((v,k) for k,v in dict_vehicle_no_train_df.iteritems())
# final_test_df.vehicle_no = final_test_df.vehicle_no.replace(rev_dict_vehicle_no_test_df)

In [266]:
# ist_test = final_test_df['ist_timestamp']
# final_train_df_copy = final_train_df.copy()
# final_test_df_copy = final_test_df.copy()

In [267]:
# final_train_df.drop(['ist_timestamp','time_taken'], axis=1, inplace=True)
# final_test_df.drop(['ist_timestamp','time_taken','time_btn_tolls'], axis=1, inplace=True)

In [189]:
train_df.drop(['time_taken'], axis=1, inplace=True)
test_df.drop(['time_taken'], axis=1, inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [196]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [197]:
train_X = train_df.drop(['time_btn_tolls'], axis=1)
train_y = train_df.time_btn_tolls

In [198]:
X_train, X_test, y_train, y_test = train_test_split(train_X,train_y, test_size=0.3, random_state=42)

In [199]:
scale= MinMaxScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

In [200]:
reg_mlp = MLPRegressor(hidden_layer_sizes=(200,40), activation='logistic', alpha=8, solver='lbfgs', random_state=42)

In [203]:
# reg_svr = SVR(C=100.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1.0,
#   kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
reg_mlp.fit(X_train_scaled, y_train)
reg_mlp.score(X_test_scaled, y_test)

0.62554210319533188

In [201]:
scores = cross_val_score(reg_mlp, X_train_scaled, y_train, scoring='r2', n_jobs=-1)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.79 (+/- 0.16)


In [204]:
scale = MinMaxScaler()
test_X = test_df.drop(['time_btn_tolls'], axis=1)
X_predict_scaled = scale.fit_transform(test_X)
predictions = reg_mlp.predict(X_predict_scaled)

In [205]:
show_df = test_df.copy()
show_df['predicted'] = predictions

In [104]:
# show_df['predicted_timestamp']

In [206]:
# show_df_copy = show_df.copy()
show_df = show_df.reset_index(drop=True)

In [218]:
show_df

,toll_both,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,delay,time_btn_tolls,predicted,predicted_ist
0,1,1,0.00,11,5,9,89,-2.299028,33.987222,11.300757,2017-11-03 23:20:45.726980
1,2,1,93.71,15,5,9,89,-2.596667,4.460278,10.861162,2017-11-04 10:12:25.908822
2,3,1,53.26,18,5,9,89,-1.191944,2.979444,6.034956,2017-11-04 16:14:31.750488
3,4,1,56.30,20,5,9,89,-0.745833,2.153056,6.760934,2017-11-04 23:00:11.113720
4,5,1,88.58,23,5,9,89,-1.620556,2.551667,10.818146,2017-11-05 09:49:16.438198
5,6,1,83.91,3,5,9,89,-4.525278,3.979444,8.757234,2017-11-05 18:34:42.481340
6,7,1,116.37,8,5,9,89,-3.549444,5.251111,12.132454,2017-11-06 06:42:39.314653
7,8,1,26.47,9,5,9,89,-4.153056,0.725556,1.750225,2017-11-06 08:27:40.126057
8,10,1,39.33,12,5,9,89,-2.910556,3.007500,3.827296,2017-11-06 12:17:18.390824
9,11,1,132.47,17,5,9,89,20.011111,29.728056,17.245037,2017-11-07 05:32:00.524137


In [207]:
new_toll_dict['Laxmipuram']

9

In [208]:
show_df.index[show_df['toll_both'] == 10][0]

8

In [209]:
dict_last_ist = {}

# list_choice_truck = test_df.vehicle_no.unique().tolist()

loading_out = raw_input('Enter the loading out time in %Y-%m-%d %H:%M:%S format: ')
loading_out_delta = datetime.datetime.strptime(loading_out, '%Y-%m-%d %H:%M:%S')

show_df['predicted_ist'] = show_df['predicted'].apply(lambda x: datetime.timedelta(hours=x))
show_df['predicted_ist'] = show_df['predicted_ist'].cumsum()
show_df['predicted_ist'] = show_df['predicted_ist'].apply(lambda x: x+ loading_out_delta)
show_df['predict']
    
choice = True
while(choice):
    
    
    print 'Enter the latest toll both details: '
    
    b = []
    toll = raw_input('Toll both: ')
#     list_toll.append(toll)
    distance = input('Distance travelled from previous toll in km: ')
    ist = raw_input('IST Timestamp in %Y-%m-%d %H:%M:%S format: ')
    ist_delta = datetime.datetime.strptime(ist, '%Y-%m-%d %H:%M:%S')
    
    i = show_df.index[show_df['toll_both'] == new_toll_dict[toll]][0]
    
    final_show_df = show_df[i+1:]
    
    final_show_df['predicted_ist'] = final_show_df['predicted'].apply(lambda x: datetime.timedelta(hours=x))
    final_show_df['predicted_ist'] = final_show_df['predicted_ist'].cumsum()
    final_show_df['predicted_ist'] = final_show_df['predicted_ist'].apply(lambda x: x + ist_delta)
    
    final_show_df    
    
#     ist_delta 
#     b.extend([toll, vehicle, loading_out_delta.hour, ist_delta, distance, ist_delta.hour, ist_delta.weekday(), ist_delta.month,
#                   (today_date - loading_out_delta).days, (ist_delta - loading_out_delta).total_seconds()/3600])
#     a.append(b)
#     dict_last_ist[vehicle] = ist_delta
#     count+=1
    
    
#     add_df = pd.DataFrame(a, columns= ['toll_both','vehicle_no','loading_out_time', 'ist_timestamp', 'distance_travelled', 'time_of_travel',
#                                          'day_of_travel', 'month_of_travel', 'days_from_now', 'time_taken'])
    
#     add_df['time_btn_tolls'] = add_df['time_taken'].diff()
#     add_df = add_df.fillna(0)
#     final_train_df = final_train_df.append(add_df)
    
#     final_test_df = final_test_df[(final_test_df['vehicle_no'] != vehicle) | (final_test_df['toll_both'].isin(list_toll) == False)] 
    
    c = raw_input('Want to enter more vehicles?(y/n)')
    
    if (c == 'y' or c == 'Y'):
        continue
    elif (c == 'n' or c == 'N'):
        choice = False
        print 'bbye'
    else:
        print 'Wrong Choice! Exiting'
        choice = False

Enter the loading out time in %Y-%m-%d %H:%M:%S format: 2017-11-03 12:02:43
Enter the latest toll both details: 
Toll both: Manguli
Distance travelled from previous toll in km: 54
IST Timestamp in %Y-%m-%d %H:%M:%S format: 2017-11-04 12:10:12


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Want to enter more vehicles?(y/n)n
bbye


In [210]:
final_show_df['time_taken'] = final_show_df['time_btn_tolls'].cumsum()
final_show_df['predicted_time_taken'] = final_show_df['predicted'].cumsum()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [211]:
final_show_df

,toll_both,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,delay,time_btn_tolls,predicted,predicted_ist,time_taken,predicted_time_taken
3,4,1,56.30,20,5,9,89,-0.745833,2.153056,6.760934,2017-11-04 18:55:51.363232,2.153056,6.760934
4,5,1,88.58,23,5,9,89,-1.620556,2.551667,10.818146,2017-11-05 05:44:56.687710,4.704722,17.579080
5,6,1,83.91,3,5,9,89,-4.525278,3.979444,8.757234,2017-11-05 14:30:22.730852,8.684167,26.336314
6,7,1,116.37,8,5,9,89,-3.549444,5.251111,12.132454,2017-11-06 02:38:19.564165,13.935278,38.468768
7,8,1,26.47,9,5,9,89,-4.153056,0.725556,1.750225,2017-11-06 04:23:20.375569,14.660833,40.218993
8,10,1,39.33,12,5,9,89,-2.910556,3.007500,3.827296,2017-11-06 08:12:58.640336,17.668333,44.046289
9,11,1,132.47,17,5,9,89,20.011111,29.728056,17.245037,2017-11-07 01:27:40.773649,47.396389,61.291326
10,12,1,69.22,19,5,9,89,18.171944,2.031111,16.034894,2017-11-07 17:29:46.393474,49.427500,77.326220
11,14,1,144.61,1,5,9,89,16.879583,5.390000,16.524288,2017-11-08 10:01:13.831940,54.817500,93.850509
12,13,1,21.91,2,5,9,89,18.563056,0.694722,30.266520,2017-11-09 16:17:13.302659,55.512222,124.117029


In [286]:
# for num, truck in dict_vehicle_no_train_df.iteritems():
list_truck = []
for truck, ist in dict_last_ist.iteritems():
    list_truck.append(truck)
    
for truck in list_truck:
    i = rev_dict_vehicle_no_test_df[truck]
    dict_last_ist[i] = dict_last_ist.pop(truck)

In [290]:
dict_last_ist

{128: datetime.datetime(2017, 11, 4, 12, 10, 12),
 129: datetime.datetime(2017, 10, 4, 20, 40, 53),
 130: datetime.datetime(2017, 11, 23, 22, 15, 25)}

In [287]:
show_df['eta'] = show_df['predicted']

In [280]:
dict_last_ist

{'BR02GA0735': datetime.datetime(2017, 11, 4, 12, 10, 12),
 'JH02M1289': datetime.datetime(2017, 11, 23, 22, 15, 25),
 'TN28BB9075': datetime.datetime(2017, 10, 4, 20, 40, 53)}

In [291]:
show_df['time_taken'] = 0
for truck, ist in dict_last_ist.iteritems():
    show_df.loc[show_df['vehicle_no'] == truck, 'time_taken'] = show_df['predicted'][show_df['vehicle_no'] == truck].cumsum()

In [293]:
for truck, ist in dict_last_ist.iteritems():
    show_df['eta'][show_df['vehicle_no'] == truck] = str(ist)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [294]:
show_df['eta'] = show_df['eta'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

In [295]:
show_df.dtypes

toll_both                      int64
vehicle_no                     int64
loading_out_time              object
distance_travelled           float64
time_of_travel                object
day_of_travel                 object
month_of_travel               object
days_from_now                 object
predicted                    float64
eta                   datetime64[ns]
time_taken                   float64
dtype: object

In [296]:
show_df['time_taken'] = show_df['time_taken'].apply(lambda x: datetime.timedelta(hours=x) )

In [297]:
show_df['eta'] = show_df['eta'] + show_df['time_taken']

In [299]:
show_df['vehicle_no'] = show_df['vehicle_no'].apply(lambda x:dict_vehicle_no_train_df[x])

In [300]:
rev_toll_dict = dict((v,k) for k,v in new_toll_dict.iteritems())
show_df.toll_both = show_df.toll_both.replace(rev_toll_dict)

In [301]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,eta,time_taken
383,Gudipada (Old Gangapada),BR02GA0735,1,56.30,20,5,9,87,2.788219,2017-11-04 14:57:29.589585,0 days 02:47:17.589585
199,Gurapalli,BR02GA0735,1,88.58,23,5,9,87,4.217417,2017-11-04 19:10:32.290262,0 days 07:00:20.290262
2159,Bellupada,BR02GA0735,1,83.91,3,5,9,87,4.593162,2017-11-04 23:46:07.674173,0 days 11:35:55.674173
2335,Madapam,BR02GA0735,1,116.37,8,5,9,87,9.319069,2017-11-05 09:05:16.321401,0 days 20:55:04.321401
1475,Chilakapalem,BR02GA0735,1,26.47,9,5,9,87,2.555393,2017-11-05 11:38:35.735443,0 days 23:28:23.735443
2518,Nathavalasa/ Vizianagaram,BR02GA0735,1,39.33,12,5,9,87,-0.108049,2017-11-05 11:32:06.760717,0 days 23:21:54.760717
1655,Vempadu,BR02GA0735,1,132.47,17,5,9,87,12.403067,2017-11-05 23:56:17.802294,1 days 11:46:05.802294
1094,Krishnavaram,BR02GA0735,1,69.22,19,5,9,87,3.511915,2017-11-06 03:27:00.697677,1 days 15:16:48.697677
2868,Kalaparru,BR02GA0735,1,144.61,1,5,9,87,15.326998,2017-11-06 18:46:37.889529,2 days 06:36:25.889529
2695,Pottipadu,BR02GA0735,1,21.91,2,5,9,87,-1.506151,2017-11-06 17:16:15.747275,2 days 05:06:03.747275


In [302]:
show_df_copy = show_df.copy()
show_df.drop(['time_taken'], axis=1, inplace=True)

In [303]:
show_df['actual'] = ist_test

In [304]:
show_df

,toll_both,vehicle_no,loading_out_time,distance_travelled,time_of_travel,day_of_travel,month_of_travel,days_from_now,predicted,eta,actual
383,Gudipada (Old Gangapada),BR02GA0735,1,56.30,20,5,9,87,2.788219,2017-11-04 14:57:29.589585,2017-09-23 20:43:26
199,Gurapalli,BR02GA0735,1,88.58,23,5,9,87,4.217417,2017-11-04 19:10:32.290262,2017-09-23 23:16:32
2159,Bellupada,BR02GA0735,1,83.91,3,5,9,87,4.593162,2017-11-04 23:46:07.674173,2017-09-24 03:15:18
2335,Madapam,BR02GA0735,1,116.37,8,5,9,87,9.319069,2017-11-05 09:05:16.321401,2017-09-24 08:30:22
1475,Chilakapalem,BR02GA0735,1,26.47,9,5,9,87,2.555393,2017-11-05 11:38:35.735443,2017-09-24 09:13:54
2518,Nathavalasa/ Vizianagaram,BR02GA0735,1,39.33,12,5,9,87,-0.108049,2017-11-05 11:32:06.760717,2017-09-24 12:14:21
1655,Vempadu,BR02GA0735,1,132.47,17,5,9,87,12.403067,2017-11-05 23:56:17.802294,2017-09-25 17:58:02
1094,Krishnavaram,BR02GA0735,1,69.22,19,5,9,87,3.511915,2017-11-06 03:27:00.697677,2017-09-25 19:59:54
2868,Kalaparru,BR02GA0735,1,144.61,1,5,9,87,15.326998,2017-11-06 18:46:37.889529,2017-09-26 01:23:18
2695,Pottipadu,BR02GA0735,1,21.91,2,5,9,87,-1.506151,2017-11-06 17:16:15.747275,2017-09-26 02:04:59
